In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [9]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),
                          chrome_options=options)
infos_main_screen = []

for i in range(1, 12):
    url = f"https://polkastarter.com/projects?page={i}#search"
    driver.get(url)

    f = driver.find_element(
        By.XPATH, '//table[@id="ps--funded-projects__table"]//tbody')
    for name in f.find_elements_by_xpath(".//tr"):

        info = name.find_elements_by_xpath(".//td")
        temp_list = [i.text for i in info]
        infos_main_screen.append(temp_list)
        
driver.quit()




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
There is no [linux64] chromedriver for browser 100.0.4896 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_linux64.zip
Driver has been saved in cache [/home/lchaves/.wdm/drivers/chromedriver/linux64/100.0.4896.60]
/tmp/ipykernel_771/1939220154.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),
/tmp/ipykernel_771/1939220154.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),
/home/lchaves/.pyenv/versions/3.8.12/envs/web3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:426: UserWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  warnings

In [10]:
cols = [
    "Project Name", "Type", "Participants", "Total Raised", "Current Price",
    "ATH Since IDO", "Ended in (UTC)", "Networks"
]

df = pd.DataFrame(infos_main_screen, columns=cols).drop_duplicates()

In [11]:
df[['Project Name', 'Ticker']] = pd.Series(df['Project Name']).str.split("\n",
                                                              expand=True)

In [12]:
df['Link'] = "https://polkastarter.com/projects/" + (
    df['Project Name'].str.lower()).str.replace(" ", "-").str.replace(
        ".", "-")


/tmp/ipykernel_771/1308547449.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Link'] = "https://polkastarter.com/projects/" + (


In [13]:
df

,Project Name,Type,Participants,Total Raised,Current Price,ATH Since IDO,Ended in (UTC),Networks,Ticker,Link
0,Cryptoverse,LAND SALE,3628,"$3,160,540.00",-,-,March 14th 2022,,None,https://polkastarter.com/projects/cryptoverse
1,Mecha Morphing,IDO,1547,"$400,000.00",$2.4899,+843%,March 29th 2022,,$MAPE,https://polkastarter.com/projects/mecha-morphing
2,Deliq,IDO,747,"$400,000.00",$0.1062,+59%,March 23rd 2022,,$DLQ,https://polkastarter.com/projects/deliq
3,Orbitau,IDO,1021,"$245,000.00",$0.1151,+575%,February 23rd 2022,,$TAUM,https://polkastarter.com/projects/orbitau
4,Ark Rivals,IDO,1366,"$300,000.00",$0.035,+171%,February 22nd 2022,,$ARKN,https://polkastarter.com/projects/ark-rivals
...,...,...,...,...,...,...,...,...,...,...
96,Exeedme,IDO,263,"$75,000.00",$0.1949,"+3,734%",December 30th 2020,,$XED,https://polkastarter.com/projects/exeedme
97,Fire Protocol,IDO,205,"$50,000.00",$0.0236,"+2,610%",December 28th 2020,,$FIRE,https://polkastarter.com/projects/fire-protocol
98,MahaDAO,IDO,398,"$125,000.00",$3.78,"+4,201%",December 17th 2020,,$MAHA,https://polkastarter.com/projects/mahadao
99,SpiderDAO,IDO,190,"$100,000.00",$0.0022,+295%,December 15th 2020,,$SPDR,https://polkastarter.com/projects/spiderdao


In [14]:
df['Link'] = df['Link'].str.replace("polychain-monsters", "polkamon")
df['Link'] = df['Link'].str.replace("shyft-network-inc-", "shyft-network-inc")


In [18]:
df.to_excel("../raw_data/dados_pag_inicial_polka.xlsx")

In [20]:
cont_final = []
for url in df['Link']:
    print(url)
    content = requests.get(url)
    breakpoint()
    soup = BeautifulSoup(content.content, 'html.parser')
    data = soup.find("div", {"id": "app-content"}).find_next()
    dc = json.loads(data.attrs['data-project-json'])
    cont_final.append(dc)


https://polkastarter.com/projects/cryptoverse
https://polkastarter.com/projects/mecha-morphing
https://polkastarter.com/projects/deliq
https://polkastarter.com/projects/orbitau
https://polkastarter.com/projects/ark-rivals
https://polkastarter.com/projects/artmeta
https://polkastarter.com/projects/mobland
https://polkastarter.com/projects/galaxy-fight-club
https://polkastarter.com/projects/impactmarket
https://polkastarter.com/projects/crypto-fight-club
https://polkastarter.com/projects/rainmaker-games
https://polkastarter.com/projects/safle
https://polkastarter.com/projects/domi-online
https://polkastarter.com/projects/totem
https://polkastarter.com/projects/unbound-finance
https://polkastarter.com/projects/nitro
https://polkastarter.com/projects/happyland
https://polkastarter.com/projects/humans
https://polkastarter.com/projects/zodium
https://polkastarter.com/projects/dtravel
https://polkastarter.com/projects/quidd
https://polkastarter.com/projects/mytheria
https://polkastarter.com/p

In [21]:
pd.DataFrame(cont_final).drop("description", axis=1).to_excel("../raw_data/scraped_data_polka.xlsx")

In [31]:
options = Options()
# options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),
                          chrome_options=options)

all = []
for url in df['Link']:

    driver.get(url)
    tx = driver.find_element(By.XPATH, '//*[@id="description"]/div/div[1]')
    tags = tx.find_elements(By.TAG_NAME, "h2")
    tgs = [(i.text) for i in tags]

    data = {"link":url}
    if len(tgs) == 1 :
        data[tgs[0]] = begin.replace(tgs[0], "")
    elif len(tgs) == 0:
        data["No Label"] = tx.text

    else:
        begin, end = tx.text.split(tgs[1])
        data[tgs[0]] = begin.replace(tgs[0], "")

        for iten in tgs[2:]:

            begin, end = end.split(iten)
            data[iten] = begin


    all.append(data)
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/home/lchaves/.wdm/drivers/chromedriver/linux64/100.0.4896.60/chromedriver] found in cache
/tmp/ipykernel_771/2739925226.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),
/tmp/ipykernel_771/2739925226.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),


KeyError: 'Link'

In [23]:
df = pd.DataFrame(all)

In [24]:
for col in df.columns:

    df[col] = df[col].str.replace("\n", " ").str.replace("  ", " ")

In [35]:
df.to_excel("../raw_data/text_polka.xlsx", index=False)